In [1]:
import os
import pandas as pd
import numpy as np 
import plotly.graph_objects as go

from dash import Dash, dcc, html, Input, Output,callback,  dash_table
from dash import jupyter_dash
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
from dash import no_update

C:\Users\marie\AppData\Local\Temp\ipykernel_8596\1594841681.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
features_path = r"C:\Users\marie\rep_codes\udder_project\udder_processing\features_dict\gmfeature_table.csv"
data_path = r"C:\Users\marie\rep_codes\udder_project\delpro_vms\data_out"
# data

# read features
features_df = pd.read_csv(features_path)


In [3]:
# Quarter features

In [4]:
keyword_dict = [{'label':'yield', 'value':'myield'},
                {'label':'conductivity', 'value': 'mcond'},
                {'label':'peak flow', 'value':'mpf'},
                {'label':'kick offs', 'value': 'mko'}, 
                {'label':'teats not found', 'value': 'mtnf'}, 
                {'label':'incomplete', 'value': 'minc'}]
duration_dict = [{'label': 'last 7 days', 'value': '7d'},{'label': 'current lactation', 'value': 'cl'}]
sides_dict = {'front':'lf|rf', 'back':'lb|rb', 'left':'lf|lb', 'right':'rf|rb'}

In [5]:
def blank_fig():
    fig = go.Figure(go.Scatter3d(x=[], y = [], z=[]))
    fig.update_layout(paper_bgcolor="black")
    fig.update_layout(legend_font_color="white")
    fig.update_scenes(xaxis_visible=False, yaxis_visible=False,zaxis_visible=False)
    return fig

def return_data(feature):
    filename = feature + "_integrated.csv"
    file_path = os.path.join(data_path, filename)
    data_df = pd.read_csv(file_path)
    return data_df

def quarter_corr(quarter, duration, data_df):
    global features_df
    selected_features_q = features_df.filter(regex=(quarter +"|cow"))
    # selected_features_q.loc[:, selected_features_q.columns[1:]] = selected_features_q[selected_features_q.columns[1:]].apply(pd.to_numeric, errors='coerce')
    # selected_features_q = selected_features_q.groupby("cow").median().reset_index()
    selected_data_q = data_df.filter(regex=(quarter +"|cow")).filter(regex=(duration +"|cow"))
    selected_data_q = selected_data_q.merge(data_df[["cow", "dim"]], on = "cow")
    merged = selected_features_q.merge(selected_data_q, on = "cow", how = "left")
    corr = merged.drop("cow", axis = 1).corr(method = 'pearson')
    return corr

def side_corr(side, duration, data_df):
    global features_df, sides_dict
    quarter1 = sides_dict[side]
    selected_features_q = features_df.filter(regex=(side +"|cow"))
    # selected_features_q.loc[:, selected_features_q.columns[1:]] = selected_features_q[selected_features_q.columns[1:]].apply(pd.to_numeric, errors='coerce')
    # selected_features_q = selected_features_q.groupby("cow").median().reset_index()
    selected_data_q = data_df.filter(regex=(quarter1 +"|cow")).filter(regex=(duration +"|cow"))
    selected_data_q = selected_data_q.merge(data_df[["cow", "dim"]], on = "cow")
    merged = selected_features_q.merge(selected_data_q, on = "cow", how = "left")
    corr = merged.drop("cow", axis = 1).corr(method = 'pearson')
    return corr

def remove_outliers(data):
    iqr = []
    

def heat_plot(corr):
    fig = go.Figure(data=go.Heatmap(
                   z=corr,
                   x=corr.index,
                   y=corr.columns,
                   hoverongaps = False, colorbar=dict(tickvals=[-1,-0.5, -0.25,0,0.25, 0.5, 0.75, 1])))
    fig.update_layout(paper_bgcolor="black", font_color = "white", plot_bgcolor = "black")
    return fig

In [6]:
# the style arguments for the sidebar. We use position:fixed and a fixed width
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "16rem",
    "padding": "2rem 1rem",
}

# the styles for the main content position it to the right of the sidebar and
# add some padding.
CONTENT_STYLE = {
    "margin-left": "18rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
}

MENU_STYLE = {
    'backgroundColor': 'black',
    'color': 'white',
}

sidebar = html.Div(
    [
        html.H2("Udder", className="display-4"),
        html.Hr(),
        html.P(
            "choose a VMS variable and a duration", className="lead"
        ),
        html.Label("Feature:"),
        dcc.Dropdown(id='kw-dpdn',options= keyword_dict, value = 'yield', style = MENU_STYLE),
        dcc.RadioItems(id = 'd-button', options=duration_dict, value='7d'),
    ],
    style=SIDEBAR_STYLE,
)


In [7]:
content = html.Div(
[html.Div(
             [dbc.Row(
                [dbc.Col([dcc.Graph(id='graph1', figure = blank_fig())], md = 6),
                 dbc.Col([dcc.Graph(id='graph2', figure = blank_fig())], md = 6),]),
              dbc.Row(
                [dbc.Col([dcc.Graph(id='graph3', figure = blank_fig())], md = 6),
                 dbc.Col([dcc.Graph(id='graph4', figure = blank_fig())], md = 6),]), 
             dbc.Row(
                [dbc.Col([dcc.Graph(id='graph5', figure = blank_fig())], md = 6),
                 dbc.Col([dcc.Graph(id='graph6', figure = blank_fig())], md = 6),]), 
             dbc.Row(
                [dbc.Col([dcc.Graph(id='graph7', figure = blank_fig())], md = 6),
                 dbc.Col([dcc.Graph(id='graph8', figure = blank_fig())], md = 6),]),
             dbc.Row(
                [dbc.Col([dcc.Graph(id='graph9', figure = blank_fig())], md = 6),
                 dbc.Col([dcc.Graph(id='graph10', figure = blank_fig())], md = 6),])],)
], id="page-content", style=CONTENT_STYLE)

app = Dash(__name__, external_stylesheets=[dbc.themes.DARKLY])
app.layout = html.Div([dcc.Location(id="url"), sidebar, content])

In [8]:
@app.callback(
    Output("graph1", "figure"), 
    Input('kw-dpdn', 'value'), 
    Input('d-button', 'value'))
def update_bar_chart(feature, duration):
    quarter = 'rf'
    data_df = return_data(feature)
    lf_corr = quarter_corr(quarter, duration, data_df)
    fig = heat_plot(lf_corr)
    fig.update_layout(title=dict(text="Right front"))
    return fig

@app.callback(
    Output("graph2", "figure"), 
    Input('kw-dpdn', 'value'), 
    Input('d-button', 'value'))
def update_bar_chart(feature, duration):
    quarter = 'lf'
    data_df = return_data(feature)
    lf_corr = quarter_corr(quarter, duration, data_df)
    fig = heat_plot(lf_corr)
    fig.update_layout(title=dict(text="Left front"))
    return fig

@app.callback(
    Output("graph3", "figure"), 
    Input('kw-dpdn', 'value'), 
    Input('d-button', 'value'))
def update_bar_chart(feature, duration):
    quarter = 'rb'
    data_df = return_data(feature)
    lf_corr = quarter_corr(quarter, duration, data_df)
    fig = heat_plot(lf_corr)
    fig.update_layout(title=dict(text="Right back"))
    return fig

@app.callback(
    Output("graph4", "figure"), 
    Input('kw-dpdn', 'value'), 
    Input('d-button', 'value'))
def update_bar_chart(feature, duration):
    quarter = 'lb'
    data_df = return_data(feature)
    lf_corr = quarter_corr(quarter, duration, data_df)
    fig = heat_plot(lf_corr)
    fig.update_layout(title=dict(text="Left back"))
    return fig

@app.callback(
    Output("graph5", "figure"), 
    Input('kw-dpdn', 'value'), 
    Input('d-button', 'value'))
def update_bar_chart(feature, duration):
    side = 'front'
    data_df = return_data(feature)
    lf_corr = side_corr(side, duration, data_df)
    fig = heat_plot(lf_corr)
    fig.update_layout(title=dict(text="Front"))
    return fig

@app.callback(
    Output("graph6", "figure"), 
    Input('kw-dpdn', 'value'), 
    Input('d-button', 'value'))
def update_bar_chart(feature, duration):
    side = 'back'
    data_df = return_data(feature)
    lf_corr = side_corr(side, duration, data_df)
    fig = heat_plot(lf_corr)
    fig.update_layout(title=dict(text="Back"))
    return fig

@app.callback(
    Output("graph7", "figure"), 
    Input('kw-dpdn', 'value'), 
    Input('d-button', 'value'))
def update_bar_chart(feature, duration):
    side = 'right'
    data_df = return_data(feature)
    lf_corr = side_corr(side, duration, data_df)
    fig = heat_plot(lf_corr)
    fig.update_layout(title=dict(text="Right"))
    return fig

@app.callback(
    Output("graph8", "figure"), 
    Input('kw-dpdn', 'value'), 
    Input('d-button', 'value'))
def update_bar_chart(feature, duration):
    side = 'left'
    data_df = return_data(feature)
    lf_corr = side_corr(side, duration, data_df)
    fig = heat_plot(lf_corr)
    fig.update_layout(title=dict(text="Left"))
    return fig

@app.callback(
    Output("graph9", "figure"), 
    Input('kw-dpdn', 'value'), 
    Input('d-button', 'value'))
def update_bar_chart(feature, duration):
    quarter = 'udder'
    data_df = return_data(feature)
    lf_corr = quarter_corr(quarter, duration, data_df)
    fig = heat_plot(lf_corr)
    fig.update_layout(title=dict(text="Udder"))
    return fig


In [9]:
if __name__ == '__main__':
    app.run(jupyter_mode="tab")

OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run_server.